In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor,IsolationForest,ExtraTreesRegressor,GradientBoostingRegressor

from sklearn import preprocessing
import scoring as score # para hacer los reportes de puntajes
from time import time

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/properati_final_y_descripcion.csv')
zonas = pd.read_csv('datos/todos_los_barrios.csv')

tipo_puntaje="r2"
iteraciones_de_searchs=20
cv=5

<IPython.core.display.Javascript object>

# Preprocesamiento

In [2]:
#paso features con strings a numericos
le_zona = preprocessing.LabelEncoder()
properati['state_name'] =le_zona.fit_transform(properati['state_name'])

le_barrio = preprocessing.LabelEncoder()
le_barrio.fit(zonas.values)
properati['place_name'] = le_barrio.transform(properati['place_name'])

le_tipo = preprocessing.LabelEncoder()
properati['property_type'] = le_tipo.fit_transform(properati['property_type'])

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [3]:
properati=properati.loc[properati.price_aprox_usd<1e6]
properati=properati.loc[properati.surface_covered_in_m2<7000]
print len(properati)
properati=properati.groupby("place_name").filter(lambda x: len(x)>5)
print len(properati)

133183
132784


In [9]:
#preparo set de datos
#X = zip(properati['surface_total_in_m2'],\
       # properati['surface_covered_in_m2'],properati['place_name'],properati["cochera"],properati["lavadero"],properati["patio"],properati["terraza"],properati["hall"]\
        #,properati["lavadero"])
        #properati['surface_covered_in_m2'],properati['place_name'])
originales=["surface_total_in_m2","surface_covered_in_m2","place_name"]
features_nuevas=["cochera","comedor","living","amplio","suite","lavadero","terraza","hall","placard","balcon","parrilla","aire","patio","muebles","pileta"\
                ,"mesada","parque","quincho","escalera","chalet","monoambiente","porcelanato","caldera","solarium","duplex","cancha"]
columnas=originales+features_nuevas
X= properati.loc[:,columnas]    
y = properati["price_aprox_usd"]
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132784 entries, 0 to 136351
Data columns (total 29 columns):
surface_total_in_m2      132784 non-null float64
surface_covered_in_m2    132784 non-null float64
place_name               132784 non-null int64
cochera                  132784 non-null bool
comedor                  132784 non-null bool
living                   132784 non-null bool
amplio                   132784 non-null bool
suite                    132784 non-null bool
lavadero                 132784 non-null bool
terraza                  132784 non-null bool
hall                     132784 non-null bool
placard                  132784 non-null bool
balcon                   132784 non-null bool
parrilla                 132784 non-null bool
aire                     132784 non-null bool
patio                    132784 non-null bool
muebles                  132784 non-null bool
pileta                   132784 non-null bool
mesada                   132784 non-null bool
parque  

### Fitlrador ya no funciona

In [22]:
a_filtrar = originales+["price_aprox_usd"]
Z = properati.loc[:,a_filtrar]

filtrador = IsolationForest(n_jobs=-1,n_estimators=50)
filtrador.fit(Z)

print len(Z)
print(len(filtrador.predict(Z)==1))


132784
132784
28


# XGBoost

In [10]:
%%notify
from xgboost import XGBRegressor
xgb= XGBRegressor(nthread=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

learning_rate=np.arange(0.3,0.9,0.2)
gamma = np.arange(0.1,0.9,0.1)
subsample = [1]

param_grid = {"learning_rate": learning_rate,'min_child_weight':[1,3],'gamma':gamma,'subsample':subsample}

search = RandomizedSearchCV(xgb, param_distributions=param_grid ,cv=cv,n_iter=iteraciones_de_searchs,scoring=tipo_puntaje) 
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 220.83 segundos para 20 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.814 (std: 0.001)
Promedio validation score: 0.798 (std: 0.004)
Promedio fit time: 2.153s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.69999999999999996, 'gamma': 0.59999999999999998, 'min_child_weight': 3}

Puesto: 1
Promedio training score: 0.814 (std: 0.001)
Promedio validation score: 0.798 (std: 0.004)
Promedio fit time: 2.010s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.69999999999999996, 'gamma': 0.5, 'min_child_weight': 3}

Puesto: 1
Promedio training score: 0.814 (std: 0.001)
Promedio validation score: 0.798 (std: 0.004)
Promedio fit time: 1.744s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.69999999999999996, 'gamma': 0.40000000000000002, 'min_child_weight': 3}

Puesto: 4
Promedio training score: 0.815 (std: 0.003)
Promedio validation score: 0.798 (std: 0.004)
Promedio fit time: 1.723s
Hiper-parametros: {'subsample': 1, 'learning_rate': 0.900

<IPython.core.display.Javascript object>

In [11]:
xgboost = search.best_estimator_

# RF

In [13]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

criterion = ['mse']
max_features =["auto",None]
n_estimators = np.arange(40,100,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=cv,n_iter=iteraciones_de_searchs,scoring=tipo_puntaje) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 1119.57 segundos para 20 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.821 (std: 0.004)
Promedio fit time: 14.444s
Hiper-parametros: {'n_estimators': 95, 'max_features': 'auto', 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.821 (std: 0.003)
Promedio fit time: 13.192s
Hiper-parametros: {'n_estimators': 90, 'max_features': 'auto', 'criterion': 'mse'}

Puesto: 3
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.820 (std: 0.003)
Promedio fit time: 11.200s
Hiper-parametros: {'n_estimators': 85, 'max_features': None, 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.820 (std: 0.003)
Promedio fit time: 12.148s
Hiper-parametros: {'n_estimators': 95, 'max_features': None, 'criterion': 'mse'}

Puesto: 5
Promedio training score: 0.973 (std: 0.000)
Promedio validation score: 0.82

<IPython.core.display.Javascript object>

In [14]:
rf= search.best_estimator_

In [15]:
rf.feature_importances_

array([  8.17441973e-02,   6.25368372e-01,   1.11166586e-01,
         1.51355090e-02,   6.09481442e-03,   4.90514159e-03,
         6.02154820e-03,   4.42486654e-02,   5.61648298e-03,
         8.05274279e-03,   6.09697320e-03,   4.75145870e-03,
         4.29987434e-03,   1.12289113e-02,   6.00861089e-03,
         9.76118265e-03,   4.44102030e-03,   6.32260131e-03,
         3.35512972e-03,   4.82324102e-03,   5.40928603e-03,
         3.45216402e-03,   4.90911461e-03,   4.66610937e-04,
         2.46695049e-03,   2.52531413e-03,   5.88490637e-03,
         2.51211469e-03,   2.93047583e-03])

# ETR

In [16]:
%%notify

etr = ExtraTreesRegressor(n_jobs=-1)

n_estimators = np.arange(40,100,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"n_estimators":n_estimators}

search = GridSearchCV(etr, param_grid=param_grid ,cv=cv,scoring=tipo_puntaje) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 660.74 segundos para 12 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.787 (std: 0.008)
Promedio fit time: 12.222s
Hiper-parametros: {'n_estimators': 95}

Puesto: 2
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.786 (std: 0.008)
Promedio fit time: 11.851s
Hiper-parametros: {'n_estimators': 85}

Puesto: 3
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.786 (std: 0.008)
Promedio fit time: 12.348s
Hiper-parametros: {'n_estimators': 90}

Puesto: 4
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.785 (std: 0.009)
Promedio fit time: 10.908s
Hiper-parametros: {'n_estimators': 75}

Puesto: 5
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.785 (std: 0.008)
Promedio fit time: 11.122s
Hiper-parametros: {'n_estimators': 80}

Puesto: 6
Promedio training score: 0.998 (std: 0.000)
Promedio validation score: 0.785 (std

<IPython.core.display.Javascript object>

In [17]:
etr= search.best_estimator_

In [18]:
etr.feature_importances_

array([ 0.07987249,  0.30023241,  0.05795849,  0.01637123,  0.00947262,
        0.009189  ,  0.01400762,  0.29010424,  0.01499539,  0.01435476,
        0.02494927,  0.0105751 ,  0.00921682,  0.01494102,  0.01155181,
        0.01173841,  0.00925077,  0.01342608,  0.0082707 ,  0.01036789,
        0.01384984,  0.0070396 ,  0.00850296,  0.00885769,  0.00530134,
        0.0060109 ,  0.00906372,  0.0047486 ,  0.00577922])

# MLP

In [127]:
from sklearn.neural_network import MLPRegressor

In [128]:
%%notify

mlp = MLPRegressor()

#activation = ["identity", "logistic", "tanh", "relu"]
activation = ["identity"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"activation": activation}

search = GridSearchCV(mlp, param_grid=param_grid ,cv=cv,scoring=tipo_puntaje) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 64.37 segundos para 1 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: -48649291414.936 (std: 1278210409.982)
Promedio validation score: -48843115430.570 (std: 5226556298.293)
Promedio fit time: 10.628s
Hiper-parametros: {'activation': 'identity'}



<IPython.core.display.Javascript object>

In [129]:
mlp=search.best_estimator_

In [130]:
search.score(X_test,y_test)

-43791592775.715477

# Ensamble

In [19]:
from mlxtend.regressor import StackingRegressor

stregr = StackingRegressor(regressors=[xgb,etr], 
                           meta_regressor=rf)

In [20]:
%%notify
stregr.fit(X_train,y_train)

StackingRegressor(meta_regressor=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=95, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
         regressors=[XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight...stimators=95, n_jobs=-1,
          oob_score=False, random_state=None, verbose=0, warm_start=False)],
         verbose=0)

<IPython.core.display.Javascript object>

In [21]:
stregr.score(X_test,y_test)

0.79930132987382674

In [22]:
errores = rf.predict(X_test)-y_test
errores_porc = 100*((rf.predict(X_test)-y_test))/y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print ("Con % \nError maximo:{0}%\nError minimo:{1}%".format( max(abs(errores_porc)),min(abs(errores_porc))))
print(errores)

Error maximo:620042.105263
Error minimo:0.0
Con % 
Error maximo:10350.7878231%
Error minimo:0.0%
126407     -5541.804511
74697      25557.894737
61013      10127.368421
126493      5224.435088
130310    -53305.663158
73061       1747.368421
107737     -1442.198868
114240     27273.684211
104066    -11489.868324
32375       8641.052632
124754    -33292.631579
94633      -3235.639098
121148     -3353.684211
3694        8321.052632
60534       1326.315789
69021      -9381.498947
133444     -6094.736842
12503      -2326.315789
42819      -1136.842105
22422     -33978.947368
100531      2787.943222
36305     -78311.063815
107988     17819.458596
61641       1989.473684
51318      11927.438596
18873      67221.052632
15934     120592.982456
10525      24601.726316
16621      82880.754887
70431     -15505.263158
              ...      
24022      -1147.368421
97378      18445.473684
135970      6685.087719
113740     17912.207895
117304     -7213.978947
83344     -30578.947368
73580        17

In [23]:
count_max=0
max_error=100
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 256 de 26557 que tienen mas de 100 % de error (0%)


In [24]:
count_max=0
max_error=10
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 5372 de 26557 que tienen mas de 10 % de error (20%)


In [25]:
count_max=0
max_error=5
lista=[]
for error in errores_porc:
    if error>max_error:
        count_max+=1
        lista.append(abs(errores_porc))
print ("Hay {0} de {1} que tienen mas de {2} % de error ({3}%)".format(count_max,len(errores_porc),max_error,100 * count_max/len(errores_porc)))

Hay 8223 de 26557 que tienen mas de 5 % de error (30%)


### Errores arriba del maximo. No esta funcionando se cuelga todo

In [ ]:
# the histogram of the data
plt.figure(figsize=(12,8))
plt.hist(lista, 500, facecolor='blue')
plt.xlabel('Errores')
plt.ylabel('Cantidad')
#plt.xlim(-10000000, 10000000) #para variar el "zoom a 0"
plt.yscale('log')
plt.show()

### Errores totales

In [ ]:
# the histogram of the data
plt.figure(figsize=(12,8))
plt.hist(errores, 500, facecolor='blue')
plt.xlabel('Errores')
plt.ylabel('Cantidad')
#plt.xlim(-10000000, 10000000) #para variar el "zoom a 0"
plt.yscale('log')
plt.xscale('log')
plt.show()

# Kaggle

In [69]:
test_samples=pd.read_csv("datos/properati_dataset_testing_noprice_acondicionado.csv")
test_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        14166 non-null float64
surface_covered_in_m2      14166 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [70]:
test= zip(test_samples['surface_total_in_m2'],\
        test_samples['surface_covered_in_m2'],le_barrio.transform(test_samples['place_name']))

In [71]:
resultados = stregr.predict(test)
result=pd.DataFrame({'id':test_samples['id'],'price_usd':resultados})
result.to_csv('datos/resultados.csv',index=False)